In [ ]:
import pandas as pd
import re
import string
import collections
from nltk import word_tokenize, wordpunct_tokenize
import itertools
import nltk
import numpy as np
import requests
from bs4 import BeautifulSoup
import json

#nltk.download('punkt')

In [ ]:
import requests
from bs4 import BeautifulSoup

YourURL = 'http://www.siberianlanguages.surrey.ac.uk/audio/a-calm-year/'
r = requests.get(YourURL)

soup = BeautifulSoup(r.content)

In [ ]:
def get_all_content_parts(soup):
  all_content = soup.find_all('div', {"class": "tier"}) 
  all_content_parts = []
  part = []
  for c in all_content:
    if c.attrs['data-tier']=='ts_content_forestenets':
      all_content_parts.append(part)
      part = []
      part.append(c)
    else:
      part.append(c)
  all_content_parts.append(part)
  print(all_content_parts)
  return all_content_parts[1:]

In [ ]:
def make_all_sentence_df(all_content_parts):
  index = [i for i in range(len(all_content_parts))]
  result = pd.DataFrame(index=index, columns=['sentence','morphemes','glosses_en', 'glosses_ru', 'translation_en', 'translation_ru'])
  for i in index:
    for c in all_content_parts[i]:
      if c.attrs['data-tier']=='ts_content_forestenets':
        result['sentence'][i] = c.string
      if c.attrs['data-tier']=='ts_content_morph':
        result['morphemes'][i] = c.string
      if c.attrs['data-tier']=='ts_content_igt':
        result['glosses_en'][i] = c.string 
      if c.attrs['data-tier']=='ts_content_igt_ru':
        result['glosses_ru'][i] = c.string    
      if c.attrs['data-tier']=='ts_content_eng':
        result['translation_en'][i] = c.string
      if c.attrs['data-tier']=='ts_content_rus':
        result['translation_ru'][i] = c.string
  print(result)
  return result

In [ ]:
def string_into_tokens(s):
    wo_brackets = [i for i in s.split() if '[' not in s]
    ll = [[word_tokenize(w), ' '] for w in wo_brackets]
    s_list = list(itertools.chain(*list(itertools.chain(*ll))))
    return s_list

In [ ]:
def make_sentence_words_arr(all_sentences, i):
    #print(i)
    wf_arr = []
    tokens = string_into_tokens(all_sentences['sentence'][i])
    #print(tokens)
    #print(all_sentences['morphemes'][i])
    try:
      morphemes = all_sentences['morphemes'][i].split()
    except AttributeError:
      morphemes = np.nan
    #print(morphemes)
    #print(len(morphemes))
    try:
      glosses_en = all_sentences['glosses_en'][i].split()
    except AttributeError:
      glosses_en = np.nan
    try:
      glosses_ru = all_sentences['glosses_ru'][i].split()
    except:
      glosses_ru = np.nan
    j = 0
    off_start = 0
    sentence_index = 0
    for k in range(len(tokens)):
        if tokens[k] == ' ':
            off_start += len(tokens[k])
            continue
        elif tokens[k] in string.punctuation:
            wf_arr.append(collections.OrderedDict({
                'wf': tokens[k],
                'wtype': 'punct',
                'off_start': off_start,
                'off_end': off_start + len(tokens[k]),
                'next_word': sentence_index+1,
                'sentence_index': sentence_index
                #'ana': False
            }))
            sentence_index += 1
        else:
            try:
              glosses_en_j=glosses_en[j]
            except:
              glosses_en_j=''  
            try:
              glosses_ru_j=glosses_ru[j]
            except:
              glosses_ru_j=''
            try:
              morphemes_j=morphemes[j]
            except:
              morphemes_j=''            
            wf_arr.append(collections.OrderedDict({
                'wf': tokens[k],
                'wtype': 'word',
                'off_start': off_start,
                'off_end': off_start + len(tokens[k]),
                'next_word': sentence_index+1,
                'sentence_index': sentence_index,
                'ana': [{'glosses_ru': glosses_ru_j, 'glosses_en': glosses_en_j, 'morphemes': morphemes_j}]
                #"ana": [{"lex": "ojorsp", "gr.pos": "V", "gr.pers": "2", "gr.tense": "t1"}]
            }))
            j += 1
            sentence_index += 1
        off_start += len(tokens[k])
    return wf_arr


In [ ]:
def make_json(soup):
  all_sentences = []
  all_sentences_df = make_all_sentence_df(get_all_content_parts(soup))
  for i in range(len(all_sentences_df)):
    all_sentences.append(collections.OrderedDict({
              'text': all_sentences_df['sentence'][i],
              'words': make_sentence_words_arr(all_sentences_df, i),
              'lang': 1
          }))
  one_text = {
    "sentences": all_sentences,
    "meta": {"author": 'author', "title": soup.title.string, "year": '', "id": 1}
  }
  return one_text

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def get_all_urls(main_url):
  all_urls = []
  r = requests.get(main_url)
  soup = BeautifulSoup(r.content)
  titles = soup.find_all('h1', {'class': "entry-title"})
  for title in titles:
    all_urls.append(next(title.children).attrs['href'])
  return all_urls


In [ ]:
!mkdir 'forest-enets'

In [ ]:
main_url = 'http://www.siberianlanguages.surrey.ac.uk/language/forest-enets/?post_types=ava,avv'
all_urls = get_all_urls(main_url)
for url in all_urls:
  r = requests.get(url)
  soup = BeautifulSoup(r.content)
  text_json = make_json(soup)
  name = 'forest-enets/' + soup.title.string + '.json'
  with open(name, 'w', encoding='utf-8') as f:
      json.dump(text_json, f, ensure_ascii=False, indent=4)

In [ ]:
print('ura2')

ura2


In [ ]:
!zip -r forest-enets.zip forest-enets

  adding: forest-enets/ (stored 0%)
  adding: forest-enets/A clairvoyant - Endangered Languages and Cultures of Siberia.json (deflated 95%)
  adding: forest-enets/The one legged woman - Endangered Languages and Cultures of Siberia.json (deflated 95%)
  adding: forest-enets/A calm year - Endangered Languages and Cultures of Siberia.json (deflated 94%)
  adding: forest-enets/Little red riding hood - Endangered Languages and Cultures of Siberia.json (deflated 94%)
  adding: forest-enets/The small bear cub - Endangered Languages and Cultures of Siberia.json (deflated 93%)
  adding: forest-enets/Duck's nest - Endangered Languages and Cultures of Siberia.json (deflated 93%)
  adding: forest-enets/Fish and chitchat (5) - Endangered Languages and Cultures of Siberia.json (deflated 94%)
  adding: forest-enets/The bears, the food sledge and the helicopter - Endangered Languages and Cultures of Siberia.json (deflated 94%)
  adding: forest-enets/Fish and chitchat (2) - Endangered Languages and Cul